In [18]:
import pandas as pd
import yfinance as yf
from scipy.stats import linregress
import numpy as np
import os

def fetch_sp500_tickers():
    """Fetches the list of S&P 500 tickers."""
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp500_list = pd.read_html(url)[0]
    return sp500_list['Symbol'].values.tolist()

def fetch_and_save_weekly_stock_data(ticker, data_dir='stock_data'):
    """Fetches historical weekly stock data and saves it to a CSV file."""
    filepath = f"{data_dir}/{ticker}_weekly.csv"
    if not os.path.exists(filepath):  # Fetch and save if file doesn't exist
        stock = yf.Ticker(ticker)
        data = stock.history(period="5y", interval="1wk")
        if not os.path.exists(data_dir):
            os.makedirs(data_dir)
        data.to_csv(filepath)
    data = pd.read_csv(filepath, index_col='Date', parse_dates=True)
    return data

def calculate_and_save_normalized_ma(ticker, window=30, ma_dir='ma_data'):
    ma_filepath = os.path.join(ma_dir, f"{ticker}_30wk_MA.csv")

    # Read stock data from csv or download from Internet
    data = fetch_and_save_weekly_stock_data(ticker, data_dir='stock_data')

    # Calculate 30-week MA
    data['30_MA'] = data['Close'].rolling(window=window).mean()

    # Normalize the 30-week MA
    historical_high = data['30_MA'].max()
    data['Normalized_30_MA'] = data['30_MA'] / historical_high

    if not os.path.exists(ma_dir):
        os.makedirs(ma_dir)

    # Save both the 30-week MA and the Normalized 30-week MA to CSV
    data[['30_MA', 'Normalized_30_MA']].to_csv(ma_filepath)

    return data

def calculate_slope(series, slope_window):
    """Calculates the slope of the MA series using linear regression for the last 30 values."""
    print(series)
    print(slope_window)
    y = series.dropna().values[-slope_window:]  # Use only the last 4 non-NA values
    print(y)
    x = np.arange(len(y))
    slope, intercept, _, _, _ = linregress(x, y)
    return slope

def calculate_normalized_slop(ticker, slope_window=4):
    """Fetch the normalized ma data and find the slope."""
    data = calculate_and_save_normalized_ma(ticker)
    # print(data)

    # Calculate the slope of the MA
    slope = calculate_slope(data['Normalized_30_MA'],slope_window)

    return slope

def process_stocks(tickers):
    """Processes stocks to fetch data, calculate MA, save both, and determine slopes for ranking."""
    stock_slopes = []
    for ticker in tickers:
        slope = calculate_normalized_slop(ticker)
        stock_slopes.append((ticker, slope))
    # Sort and select top 10 stocks based on the absolute value of their slope
    top_stocks_by_slope = sorted(stock_slopes, key=lambda x: abs(x[1]), reverse=True)[:10]
    return top_stocks_by_slope

# Execution
sp500_tickers = fetch_sp500_tickers()
top_stocks_by_slope = process_stocks(["AAPL"])

# Output the top 10 stocks with the steepest MA slopes
for stock, slope in top_stocks_by_slope:
    print(f"Ticker: {stock}, MA Slope: {slope}")


Date
2019-01-28 00:00:00-05:00         NaN
2019-02-04 00:00:00-05:00         NaN
2019-02-11 00:00:00-05:00         NaN
2019-02-18 00:00:00-05:00         NaN
2019-02-25 00:00:00-05:00         NaN
                               ...   
2024-01-01 00:00:00-05:00    0.998757
2024-01-08 00:00:00-05:00    0.999027
2024-01-15 00:00:00-05:00    1.000000
2024-01-22 00:00:00-05:00    0.999813
2024-01-29 00:00:00-05:00    0.999214
Name: Normalized_30_MA, Length: 262, dtype: float64
4
[0.99902723 1.         0.99981302 0.99921378]
Ticker: AAPL, MA Slope: 3.726810782218592e-05


In [3]:
print(len(stage_2_stocks))

257
